# Obtaining the Dataset: Cora

PyTorch Geometric is an extension library to the popular deep learning framework PyTorch, and consists of various methods and utilities to ease the implementation of Graph Neural Networks.

In this hands-on exercise, I will demonstrate the training and testing of a Graph Neural Network (GNN). To demonstrate, I make use of the Cora dataset (available in PyTorch Geometric framework), which is a citation network where nodes represent documents. Each node is described by a 1433-dimensional bag-of-words feature vector. Two documents are connected if there exists a citation link between them. The ground-truth labels of only a small subset of nodes are given. The task is to infer the labels (7 in total) for all the remaining nodes (**transductive learning**). This dataset was first introduced by [Yang et al. (2016)](https://arxiv.org/abs/1603.08861) as one of the datasets of the Planetoid benchmark suite. The other citation network datasets present in Planetoid are "CiteSeer" and "PubMed".

In [14]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of classes: {dataset.num_classes}')#number of node classes

Dataset: Cora():
Number of graphs: 1
Number of classes: 7


PyTorch Geometric provides a **Data** class. An object of the **Data** class is a homogeneous graph. Such an object can hold node-level, link-level and graph-level attributes. 

In general, **Data** tries to mimic the behavior of a regular Python dictionary. 

Some of the commonly useful properties of this class and its objects are:
1. **num_node_features**: int, used in defining the 
2. **num_nodes**: int
3. **num_edge_features**: int
4. **num_edges**: int
5. **num_classes**: int

6. num_edge_types: int
7. num_node_types: int

Some of the useful methods of this class and its objects are:
1. .is_undirected()
2. .has_self_loops()

In [9]:
for (i,graph) in zip(range(len(dataset)),dataset):
        print(f'{i}-th graph: {graph}')
        print(f'------------')
        print(f'Number of nodes: {graph.num_nodes}')
        print(f'Number of node features: {graph.num_node_features}')
        print(f'Number of edges: {graph.num_edges}')
        print(f'Number of edge features: {graph.num_edge_features}')
        print(f'Average node degree: {graph.num_edges / graph.num_nodes:.2f}')
        print(f'Number of training nodes: {graph.train_mask.sum()}')
        print(f'Number of validation nodes: {graph.val_mask.sum()}')
        print(f'Number of test nodes: {graph.test_mask.sum()}')
        print(f'Has isolated nodes(nodes without edges): {graph.has_isolated_nodes()}')
        print(f'Has self-loops: {graph.has_self_loops()}')
        print(f'Is undirected: {graph.is_undirected()}')

0-th graph: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
------------
Number of nodes: 2708
Number of node features: 1433
Number of edges: 10556
Number of edge features: 0
Average node degree: 3.90
Number of training nodes: 140
Number of validation nodes: 500
Number of test nodes: 1000
Has isolated nodes(nodes without edges): False
Has self-loops: False
Is undirected: True


**Observations:**

1. The number of training nodes is few: **140**.

2. You can see that this graph holds the attributes `val_mask` and `test_mask` denoting which nodes should be used for validation and testing.

3. Furthermore, notice the use of data transformations via `transform=NormalizeFeatures()`. It row-normalizes the bag-of-words input feature vectors.

In [132]:
dataset[0].y #contains the ground-truth label of each of the 2708 nodes in the graph

tensor([3, 4, 4,  ..., 3, 3, 3])

In [134]:
len(dataset[0].y)

2708

In [133]:
import numpy as np
len(np.unique(np.array(dataset[0].y)))#np.unique() is a utility from the numpy library that removes duplicates in a numpy array

7

In [135]:
dataset[0].x #a tensor containing the node embeddings. Each embedding corresponds to one of the 7 ground-truth labels of the nodes.

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [136]:
len(dataset[0].x)

2708

In [137]:
dataset[0].x[2700] #embeddings of the 2701th node

tensor([0.0000, 0.0000, 0.0476,  ..., 0.0000, 0.0000, 0.0000])

By printing '**edge_index**', we can understand how PyG represents graph connectivity internally. We can see that for each edge, edge_index holds a tuple of two node indices, where the first value describes the node index of the source node and the second value describes the node index of the destination node of an edge.

This representation is known as the **COO format** (co-ordinate format) commonly used for representing sparse matrices. Instead of holding the adjacency information in a dense representation, PyG represents graphs sparsely, which refers to only holding the coordinates/values for which entries are non-zero.

In [138]:
dataset[0].edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [139]:
len(dataset[0].edge_index)

2

In [5]:
import torch
torch.transpose(dataset[0].edge_index,0,1) #shows the source and the destination nodes of each edge in the graph

tensor([[   0,  633],
        [   0, 1862],
        [   0, 2582],
        ...,
        [2707,  598],
        [2707, 1473],
        [2707, 2706]])

In [141]:
dataset[0].edge_attr #Edge feature matrix (default: None)

In [142]:
dataset[0].pos #Node position matrix (default: None)

## Implementing a Graph Neural Network (GNN)

In the following cell, we implement a 2-layered GNN. Each layer performs the following graph convolution operation ([Kipf et al. (2017)](https://arxiv.org/abs/1609.02907)):

$$
\mathbf{x}_v^{(\ell + 1)} = \mathbf{W}^{(\ell + 1)} \sum_{w \in \mathcal{N}(v) \, \cup \, \{ v \}} \frac{1}{c_{w,v}} \cdot \mathbf{x}_w^{(\ell)}
$$

where, $\mathbf{W}^{(\ell + 1)}$ denotes a **trainable weight matrix** of shape `[num_output_features, num_input_features]` and $c_{w,v}$ refers to a fixed normalization coefficient for each edge.

PyG implements this layer via its [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv) class, specifically the `forward` function in it. It is executed by passing in the node-feature representation `x` and the COO graph-connectivity representation `edge_index`.

The Graph Neural Network (GNN) architecture is described in a child class `GCN` derived from the `torch.nn.Module` class of PyTorch. The `GCN` class contains the dimensions of individual convolution layers and the sequence of the convolution layers and non-linear activations (that follow each of the convolution layers). 

In [11]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):#torch.nn.Module is the base class for all neural network modules in PyTorch 
    def __init__(self):#defines the layers of the GNN and initializes them
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)#GCNConv() performs message computation, aggregation of the messages, and then, updating of the node embeddings. The 1st parameter 'number of input features per node' and the 2nd argument 'number of features per output' are provided for initializing the parameters of the class GCNConv.
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, graph):#defines the computation flow of our network. An entire graph is fed to the GNN.
        x, edge_index = graph.x, graph.edge_index#'x' represents the vector of node features and edge_index represents the adjacency matrix for connectivity

        x = self.conv1(x, edge_index)#conv1.forward() gets called here. The arguments 'x' and 'edge_index' are passed as inputs to forward().
        x = F.relu(x)#Applying Relu activation on the result of the above graph-convolution operation.
        
        x = F.dropout(x, training=self.training)#Randomly zero some of the elements of the input tensor 'x' with probability p(default: 0.5) using samples from a Bernoulli distribution.Also, the mode is set to 'training' because Dropout behaves differently during training and testing.
        
        x = self.conv2(x, edge_index)#conv2.forward() gets called here. The arguments 'x' and 'edge_index' are passed as inputs to forward().
        return F.log_softmax(x, dim=1)#Applying softmax activation on the result of the above graph-convolution operation.

Then, we choose the device on which we want to deploy the GNN and the training dataset.

A **torch.device** is an object representing the device on which a torch.Tensor is or will be allocated. The torch.device contains a device type ('cpu', 'cuda' or 'mps') 

In [144]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Next, we move the GNN parameters to the chosen device.

In [145]:
model = GCN().to(device)

Also, we move the dataset to the chosen device.

In [146]:
data = dataset[0].to(device)

## Training the GNN

Then, we choose the optimization algorithm (from the *torch.optim* package) for training/optimizing the parameters of our GNN. As an input, we provided *model.parameters()* to denote which parameters (tensors) to optimize. We also defined the decay constant and learning rate (lr).

**NOTE:** During training, the weights (parameters) of the GNN are learnt and not the embeddings of the nodes. The GNN just predicts or computes the embeddings of the nodes using *neural message passing*. The GNN just maps the input node-embeddings to new embeddings. So, the GNN can be seen as graph transformer, *i.e.*, it transforms the input graph into output graph.

In [147]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

Next, we set the mode of GCN module (torch.nn.Module) to training. '*model.train()*' simple changes the '*self.training*' flag via '*self.training = training*' recursively for all modules. 

**Note**: By default, the mode is set to training and that is why they omit '*model.train()*' call. 

In [148]:
model.train()

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

We train the GCN module for 200 epochs. 

In [149]:
for epoch in range(200):
    optimizer.zero_grad()#Initializes the gradients to zero at the beginning of each epoch
    
    out = model(data)#Calls the 'forward' method in the class GCN and stores the output/prediction of the network for all nodes in a 
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])#Calculates the negative log likelihood loss (nll) between original ('data.y') and predicted data ('out') points
    
    loss.backward()#Backward pass for computing the gradients of the loss w.r.t to learnable parameters
    optimizer.step()#Updates the learnt parameters at the end of each epoch

<span style="color:red"> **NOTE:**</span>

**In each epoch, the entire training set is forwarded through the GNN.** 

**So, in an epoch, all the graphs in the training set are given as input to `train()`.**

<span style="color:red">**In `train()`, are the graphs input as an argument to `model()` one by one? Or, is the entire training set input as an argument to `model()` at once?** </span>

**Final node-embeddings computed by the trained GNN:**

In [150]:
model(data) #model(data) is the tensor containing the node embeddings computed by the GNN

tensor([[-4.5692, -2.9545, -2.4609,  ..., -3.9787, -4.8001, -3.9018],
        [-3.5372, -3.8077, -4.6999,  ..., -0.1340, -3.4032, -4.1923],
        [-3.5429, -4.7425, -5.0581,  ..., -0.1691, -5.2615, -3.5513],
        ...,
        [-0.9781, -1.1606, -4.4688,  ..., -4.2733, -2.5117, -2.1572],
        [-3.4227, -3.6199, -4.1960,  ..., -2.3243, -6.3733, -4.8514],
        [-3.1746, -3.6696, -3.3696,  ..., -2.4823, -5.3351, -3.9233]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

## Testing the trained GNN

Next, we set the mode of GCN module (torch.nn.Module) to testing. `eval()` is a method of the `torch.nn.module` class that sets the NN module (object of the `torch.nn.module` class or of classes derived from `torch.nn.module`) to testing or evaluation mode. This is equivalent to executing: ***model.train(mode=False)***. 

In [151]:
model.eval()

GCN(
  (conv1): GCNConv(1433, 16)
  (conv2): GCNConv(16, 7)
)

In [152]:
model(data)[0]

tensor([-5.3779, -4.6365, -3.7765, -0.0811, -3.6247, -6.8765, -4.3396],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [153]:
model(data)[0].argmax(dim=0)

tensor(3, device='cuda:0')

In [100]:
model(data)[1]

tensor([-1.0887e+01, -1.3381e+01, -1.6949e+01, -1.3455e+01, -2.5868e-05,
        -1.3200e+01, -1.2966e+01], device='cuda:0', grad_fn=<SelectBackward0>)

In [154]:
model(data)[1].argmax(dim=0)

tensor(4, device='cuda:0')

In [155]:
model(data)[0:2]

tensor([[-5.3779, -4.6365, -3.7765, -0.0811, -3.6247, -6.8765, -4.3396],
        [-4.0937, -4.6657, -5.3559, -3.9433, -0.0626, -6.1017, -4.7951]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [156]:
model(data)[0:2].argmax(dim=0) #dim 0 => along each column

tensor([1, 0, 0, 0, 1, 1, 0], device='cuda:0')

In [157]:
model(data)[0:2].argmax(dim=1) #dim 1 =>along each row (feature vector)

tensor([3, 4], device='cuda:0')

Then, obtain the predicted label by storing the label corresponding to the node embedding that has max probability. Note that, as mentioned above, each of the 7 embeddings of a node corresponds to one of the 7 labels possible for a node. 

In [158]:
pred = model(data).argmax(dim=1)
pred #tensor containing list of indices of the largest features of each node in the graph

tensor([3, 4, 4,  ..., 0, 3, 3], device='cuda:0')

In [159]:
len(pred)

2708

In [160]:
data.test_mask #tensor that labels each of the 2708 nodes of the graph as a test sample or non-test sample

tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')

In [161]:
len(data.test_mask)

2708

In [162]:
pred[data.test_mask]

tensor([1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
        2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 5, 5, 5, 4, 5, 1, 1, 3, 0, 1, 1, 6,
        2, 1, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5,
        5, 5, 2, 2, 2, 2, 2, 6, 6, 3, 0, 0, 0, 0, 5, 0, 0, 0, 3, 0, 0, 6, 0, 0,
        3, 3, 3, 3, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3,
        3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 4, 4, 4, 4, 4, 3, 2, 5, 5, 5, 5,
        6, 5, 5, 5, 5, 6, 4, 4, 0, 0, 1, 0, 0, 0, 6, 6, 6, 6, 0, 6, 6, 0, 0, 0,
        0, 0, 0, 0, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        6, 6, 5, 6, 6, 5, 5, 5, 5, 0, 5, 0, 4, 0, 3, 3, 3, 2, 3, 1, 3, 3, 3, 3,
        3, 3, 1, 3, 3, 4, 4, 4, 3, 3, 3,

In [163]:
len(pred[data.test_mask])

1000

In [164]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [165]:
data.y

tensor([3, 4, 4,  ..., 3, 3, 3], device='cuda:0')

In [166]:
len(data.y)

2708

In [167]:
data.y[data.test_mask]

tensor([3, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
        2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 4, 4, 4, 4, 1, 1, 3, 1, 0, 3, 0,
        2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5,
        5, 5, 2, 2, 2, 2, 1, 6, 6, 3, 0, 0, 5, 0, 5, 0, 3, 5, 3, 0, 0, 6, 0, 6,
        3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 4, 4, 4, 0, 3, 3, 2, 5, 5, 5, 5,
        6, 5, 5, 5, 5, 0, 4, 4, 4, 0, 0, 5, 0, 0, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0,
        3, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4,
        6, 6, 5, 6, 6, 3, 5, 5, 5, 0, 5, 0, 4, 4, 3, 3, 3, 2, 2, 1, 3, 3, 3, 3,
        3, 3, 5, 3, 3, 4, 4, 3, 3, 3, 3,

In [169]:
len(data.y[data.test_mask])

1000

Compute the total number of correct predictions

In [170]:
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
correct

tensor(801, device='cuda:0')

Finally, we compute and print the accuracy performance of the GNN.

In [171]:
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8010


**NOTE:** 
Without normalizing the node features, the accuracy of the same GNN (with everything else unchanged) was 0.8070, *i.e.*, the GNN predicted the labels of 807 nodes correctly.

## Comparing the Performance of the GNN with that of a Multi-Layered Perceptron (MLP)

In theory, the category of a document can solely be based on and inferred from its content, *i.e.* it's bag-of-words feature representation, without taking any relational information into account.

Let's verify that by constructing a simple MLP that operates solely on the node features (using shared weights across all nodes):

The MLP here is defined by two linear layers and enhanced by ReLU non-linearity and dropout.
Here, at first, the 1433-dimensional feature vector is reduced to a low-dimensional embedding (`hidden_channels=16`), while the second linear layer acts as a classifier that maps each low-dimensional node embedding to one of the 7 classes.

In [7]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(dataset.num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

model = MLP(hidden_channels=16)
print(model)

MLP(
  (lin1): Linear(in_features=1433, out_features=16, bias=True)
  (lin2): Linear(in_features=16, out_features=7, bias=True)
)


Let's train this MLP using the **cross entropy loss** and **Adam optimizer**. A `test` function evaluates how well this model performs on the test-node set.

In [10]:
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(graph.x)  # Perform a single forward pass.
      loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(graph.x)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[graph.test_mask] == graph.y[graph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(graph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 1.9615
Epoch: 002, Loss: 1.9557
Epoch: 003, Loss: 1.9505
Epoch: 004, Loss: 1.9423
Epoch: 005, Loss: 1.9327
Epoch: 006, Loss: 1.9279
Epoch: 007, Loss: 1.9144
Epoch: 008, Loss: 1.9087
Epoch: 009, Loss: 1.9023
Epoch: 010, Loss: 1.8893
Epoch: 011, Loss: 1.8776
Epoch: 012, Loss: 1.8594
Epoch: 013, Loss: 1.8457
Epoch: 014, Loss: 1.8365
Epoch: 015, Loss: 1.8280
Epoch: 016, Loss: 1.7965
Epoch: 017, Loss: 1.7984
Epoch: 018, Loss: 1.7832
Epoch: 019, Loss: 1.7495
Epoch: 020, Loss: 1.7441
Epoch: 021, Loss: 1.7188
Epoch: 022, Loss: 1.7124
Epoch: 023, Loss: 1.6785
Epoch: 024, Loss: 1.6660
Epoch: 025, Loss: 1.6119
Epoch: 026, Loss: 1.6236
Epoch: 027, Loss: 1.5827
Epoch: 028, Loss: 1.5784
Epoch: 029, Loss: 1.5524
Epoch: 030, Loss: 1.5020
Epoch: 031, Loss: 1.5065
Epoch: 032, Loss: 1.4742
Epoch: 033, Loss: 1.4581
Epoch: 034, Loss: 1.4246
Epoch: 035, Loss: 1.4131
Epoch: 036, Loss: 1.4112
Epoch: 037, Loss: 1.3923
Epoch: 038, Loss: 1.3055
Epoch: 039, Loss: 1.2982
Epoch: 040, Loss: 1.2543


In [177]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.5900


As one can see, the MLP performs rather bad with only about 59% test accuracy. But why does the MLP not perform better? The main reason is that this MLP suffers from heavy overfitting due to having access to only a **few training nodes**, and therefore generalizes poorly to unseen node representations.

It also fails to incorporate an important bias into the model: **Cited papers are very likely related to the category of a document**. That is exactly where Graph Neural Networks come into play and can help to boost the performance.

So, relational information plays a crucial role in obtaining better performance.

TO DO:
----------
1. Print the evolution of weight matrix during the training of GNN. Print all parameters of importance during training of GNN.
2. Implement the optional exercise.